# 网络数据分析-数据挖掘实验报告
## 安然邮件通联关系提取
#### *王凌           2019/5/18*
---

## 一、实验目的和要求
        给出安然公司的邮件数据集，获取邮件的收发关系，发送时间、正文内容等通联关系数据。利用Pagerank算法计算通联关系中每一个人物的权重，结合网络分析软件（Gephi）建立起通联网络，进行可视化。
-------

## 二、实验流程
1. 数据格式分析
2. 通联关系网络——点的数据获取和构建
3. 通联关系网络——边的数据获取和构建
4. 一张等权重的无向通联关系图
5. 基于Pagerank算法的有向图建立
6. 人以群分——社团发现
7. 总结
---

### 2.1数据格式分析
        首先我们打开一份邮件来查看一下邮件的格式。
![邮件格式](./pics/1.png "可选标题")

        在图片中，第一个绿色框内有一部分邮件头数据，包括收发件人的邮箱、邮件发送时间、主题等信息。第二个绿色框中是正文的信息。为了构建通联关系网络，我们需要的是发件人的信息和收件人的信息。这一部分信息在每封邮件中存在于两处。
        一处在第一个绿色框中，呈现形式为发件人和收件人的邮箱地址。
        另一处在黄色框中，呈现形式为收件人和发件人的姓名。
        我采用黄色框中的数据作为构建通联关系的数据来源，不采用邮箱地址的原因很简单，看下面这个安然公司人员名单。
![人员名单](./pics/2.png"人员名单")
        
        我们要建立的是安然高层管理人员的通联关系网络，高管人员的姓名在这个文档里都已经明确写出。结合高管人员姓名的格式和黄色框中人物姓名的格式，显然采用黄色框中的姓名比起绿色框的邮箱地址更具有优势。
        那么实际上，从一封邮件中，我们已经可以获取到点和边的关系了，只要获取到这些数据，就可以构建出网络。

### 2.2通联关系网络——点的数据获取和构建
    那么我们首先从人员名单中获取这群高管的姓名吧～

In [1]:
#读取安然高层人员信息文档
#input 人员信息
#output 人员姓名
path = './enron employees.txt'
with open(path, 'r') as f:
    for each in f.readlines():
        p = each.split('\t')
        q = p[1].split('  ')
        print(q[0])

Albert Meyers
Thomas Martin
Andrea Ring
Andrew Lewis
Andy Zipper
Jeffrey Shankman
Barry Tycholiz
Benjamin Rogers
Bill Rapp
xxx

Bradley Mckay
xxx

Richard Sanders
Cara Semperger
Daron Giron
Charles Weldon
Chris Dorland
Chris Germany
xxx

Cooper Richey
Craig Dean
Dana Davis
Dan Hyvl
Danny McCarty
Daren Farmer
Darrell Schoolcraft
Daron Giron
David Delainey
Susan Bailey
xxx

Diana Scholtes
Thomas Martin
Don Baughman
Drew Fossum
James Steffes
xxx

xxx

Mark Haedicke
Elizabeth Sager
Eric Bass
Eric Saibi
Errol McLaughlin
Mark Taylor
Sandra Brawner
Larry Campbell
Peter Keavey
Fletcher Sturm
Frank Ermis
Geir Solberg
Geoffery Storey
Gerald Nemec
Greg Whalley
xxx

Harpreet Arora
Andrew Lewis
Holden Salisbury
Hunter Shively
James Derrick
James Steffes
Jane Tholt
xxx

Jason Wolfe
Jay Reitmeyer
Jeff Dasovich
Jeff King
John Hodge
Jeffrey Shankman
Jeffery Skilling
Daren Farmer
xxx

Jim Schwieger
Vince Kaminski
Vince Kaminski
Steven Kean
xxx

Joe Parks
Joe Quenet
Joe Stepenovitch
John Arnold
John Forn

        可以看到除了正确获取到的人员姓名外，还存在两种类型的错误：
        1、第一类是获取到“xxx”
        2、第二类是获取到 空行
        不过好在这类错误比例很低，手动删除即可。
        之后将所有的人员姓名按照字母顺序排序之后保存到一个csv中，命名为“new_member.csv”。
        csv格式示例如下：
|  id|
|:-:|
|Andrea Ring|
|Andrew Lewis|
|...|
|Vladi Pimenov|
        
        得到这些姓名之后，通联关系网络的“点”就已经得到了。
        下一步我们要获取所有人员的

<div align="center"> 姓 和 名 </div>
        
        这一步的意义在哪里呢？后面再说，先看结果。

In [2]:
with open('./new_member.csv', 'r') as f:
    p = f.readlines()
    p = set(p)
    p = list(p)
    dd = []
    ddd = []
    for each in p:
        x = each.split(' ')
        dd.append(x[-1][:-1])
        ddd.append(x[0])
    print(ddd)#名
    print(dd)#姓

['Mike', 'Eric', 'Vince', 'Jim', 'Kate', 'Andrea', 'Teb', 'Don', 'John', 'Philip', 'Patrice', 'Jeffery', 'Richard', 'Tana', 'Rick', 'Joe', 'Chris', 'Robert', 'Kam', 'Gerald', 'John', 'Randall', 'Andrew', 'Keith', 'Rod', 'Matthew', 'Susan', 'id\n', 'Danny', 'Barry', 'Phillip', 'Cooper', 'Jane', 'Eric', 'Greg', 'Susan', 'Richard', 'Darrell', 'Jay', 'Greg', 'Elizabeth', 'Michael', 'Susan', 'Stephanie', 'Kenneth', 'Scott', 'Chris', 'Frank', 'Kevin', 'Robert', 'Juan', 'Mike', 'Paul', 'Kevin', 'Joe', 'Steven', 'Jason', 'Jeff', 'Holden', 'Stanley', 'Martin', 'Shelley', 'David', 'James', 'Paul', 'Jeffrey', 'Marie', 'Theresa', 'Andy', 'Sandra', 'Errol', 'Hunter', 'Michelle', 'Tom', 'Fletcher', 'Dan', 'Kevin', 'Geir', 'Kim', 'Drew', 'Mark', 'Michael', 'Daren', 'John', 'Bill', 'Mark', 'Michelle', 'Thomas', 'Jeff', 'Joe', 'John', 'Lynn', 'Richard', 'Peter', 'Vladi', 'Lawrence', 'James', 'Dana', 'Sally', 'Diana', 'Phillip', 'Louise', 'Benjamin', 'Tori', 'Tracy', 'John', 'Judy', 'Kimberly', 'Larry',

---

### 2.3通联关系网络——边的数据获取和构建
    对于图中的一条边，我们需要哪些信息？
| 源 | 目标 | 权值 |
| :----:| :----: | :----: |
| 发件人 | 收件人 | 发件次数 |
    
    例如，在我们的整个邮件数据集中，A一共给B发送了k次邮件，那么就可以构建出这样的一条边的关系。
    A, B, k
    当然这是有向图的构建方式，对于无向图，可以采用一种冗余的构建方法。同样的条件A给B发送了k次邮件，那么我们得到的边的关系应该是：
    A, B, k;
    B, A, k;

    具体的流程介绍如下：
    首先是一个遍历文件和文件夹的函数，给定所有邮件的路径，就可以递归获取到所有邮件的路径。
    之前教员要求的是获取all_documents文件夹中的所有邮件，但在实际操作中我发现，仅仅获取这一个文件夹中的邮件是显然不够的。许多人员的大量邮件分布在其它的文件夹中，因此遍历所有的文件夹才是合理的。
---
```python
def file_path(filepath):
    fileNames = os.listdir(filepath)  # 获取当前路径下的文件名，返回List
    for file in fileNames:
        newDir = filepath + '/' + file # 将文件命加入到当前文件路径后面
        if os.path.isfile(newDir):  # 如果是文件
            _from, _to, _cc, _bcc = find_from_to_cc(newDir)
            summary_from_one(_from, _to, _cc, _bcc)
        else:
            file_path(newDir)                #如果不是文件，递归这个文件夹的路径
```
---
    
    可以看到，在file_path()这个函数中，还另外引用了find_from_to_cc()这个函数和summary_from_one()这个函数。先给出这两个函数的代码。
---
    
```python
def find_from_to_cc(file):
    '''
    这个函数的作用，就是对于单封邮件，获取邮件的发件人、收件人、抄送等信息，并保存到四个列表中，返回
    '''
    __from = []
    __to = []
    __cc = []
    __bcc = []
    with open(file, 'r') as f:
        #print(file)
        #解决macOS的隐藏文件.DS_Store问题
        if '.DS_Store' in file:
            return __from, __to, __cc, __bcc
        print(file)
        try:
            '''
            最基本的正则表达式匹配邮件的通联关系（出现的人员信息）
            有时候，一封邮件会发给多个人，在邮件中用逗号隔开：
            X-To: A, B
            那么这种情况，就需要对多个人进行分隔处理
            '''
            a = str(f.read())
            f.close()
            _from = re.findall("X-From: ([\s\S]*)\nX-To", a)
            _to = re.findall("X-To: ([\s\S]*)\nX-cc", a)
            _cc = re.findall("X-cc: ([\s\S]*)\nX-bcc", a)
            _bcc = re.findall("X-bcc: ([\s\S]*)\nX-Folder", a)
            #处理to cc bcc内多个人员的情况
            _to_ = _to[0].split(', ')
            _cc_ = _cc[0].split(', ')
            _bcc_ = _bcc[0].split(', ')
            '''
            还记得一开始建立的姓和名两个列表吗？
            邮件不仅仅是安然内部高层人士之间相互发送，邮件数据集中还存在大量冗余。
            高层-普通职工
            高层-外界人士
            广告邮件
            如何去除这些不需要的邮件？————根据收件人格式
            '''
            #将名称对应起来
            #发件人的名称比较规范
            from_name = _from[0].split(' ')
            xing = from_name[-1]
            ming = from_name[0]
            if xing in m.xing and ming in m.ming:
                __from.append(_from[0])
            for each in _to_:
                to_name = each.split(' ')
                x = to_name[-1]
                mm = to_name[0]
                if x in m.xing and mm in m.ming:
                    __to.append(each)
            for each in _cc_:
                cc_name = each.split(' ')
                x = cc_name[-1]
                mm = cc_name[0]
                if x in m.xing and mm in m.ming:
                    __cc.append(each)
            for each in _bcc_:
                bcc_name = each.split(' ')
                x = bcc_name[-1]
                mm = bcc_name[0]
                if x in m.xing and mm in m.ming:
                    __bcc.append(each)
        except:
            None
        return __from, __to, __cc, __bcc
```
---    
    
```python
def summary_from_one(_from, _to, _cc, _bcc):
    #print(_from,_to,_cc,_bcc)
    '''
    这里存在一个全局变量finald，是一个list，作用是保存下每一次邮件的通联关系
    例如，遍历到某一封邮件，经由find_from_to_cc()函数后，返回四个列表。
    _from = [A]
    _to = [B, C]
    _cc = [D, E]
    _bcc = [F, G]
    意思就是该邮件的发件人是A，发送给了B和C，抄送给了D, E, F, G
    那么finald中就会append这些项:
    (A, B), (A, C), (A, D), (A, E), (A, F)
    '''
    global finald
    if _from != []:
        if _to != []:
            for each in _to:
                finald.append((_from[0], each))
        if _cc != []:
            for each in _cc:
                finald.append((_from[0], each))
        if _bcc != []:
            for each in _bcc:
                finald.append((_from[0], each))
```
---


#### 基于收件人格式和“姓”“名”的匹配原则

    我们之前已经提到，所有的邮件的X-to、X-cc和X-bcc出现的人物中，只要是安然的高管（new_member.csv），那么一定是以姓名的形式呈现。
    例如这位M.Maggi
!['M.Maggi'](./pics/3.png)

    他在邮件中出现的时候，是以怎样的形式呢？
!['M.Maggi'](./pics/4.png)

    这样一下就能看的很清楚，高管的名字，在邮件内都是可以明确直接找到的。
    非高管的情况如何？
    

!['M.Maggi'](./pics/5.png)  

    
    往往是姓名-邮箱的格式，偶尔会见到只有姓名或是只有邮箱的情况。 
    
    知道了什么样子的人才是高管之后，我们就可以开始生成这些高管之间的通联关系了。这时候就需要借助之前已经生成的“姓”和“名”两个列表。
    生成的思路如下：
    1、利用find_from_to_cc()函数，提取出每一封邮件的from, to, cc, bcc保存为四个列表。
    2、分别对四个列表进行如下操作：
        2.1遍历列表中每一个元素p
        2.2元素p用空格分隔，如果出现了邮箱地址，那么p[-1]一定为地址，这类元素直接舍弃
        2.3如果元素p是一个人的姓名，根据国外名前姓后原则，检测p[0]是否在“名”列表中，p[-1]是否在“姓”列表中，如果有一项不在，则舍弃元素
        补充说明：在邮件中出现了一类人名，例如Ling A Wang，但是在高管名单中呈现为Ling Wang，这类姓名也会被保留下来
    3、将经过上述操作的四个列表作为summary_from_one()函数输入，将通联关系保存到finald列表中

In [3]:
'''
这一部分代码需要运行很长时间，原因在于需要对所有的邮件进行遍历。
这里为了节省时间，只采用很少的一部分邮件作为样例。
'''
import os
import re
import m
from collections import Counter

In [4]:
def file_path(filepath):
    fileNames = os.listdir(filepath)  # 获取当前路径下的文件名，返回List
    #print(fileNames)
    for file in fileNames:
        newDir = filepath + '/' + file # 将文件命加入到当前文件路径后面
        # if os.path.isdir(newDir): # 如果是文件夹
        if os.path.isfile(newDir):  # 如果是文件
            #find_from_to_cc(newDir)
            _from, _to, _cc, _bcc = find_from_to_cc(newDir)
            summary_from_one(_from, _to, _cc, _bcc)
        else:
            file_path(newDir)                #如果不是文件，递归这个文件夹的路径

In [5]:
def summary_from_one(_from, _to, _cc, _bcc):
    #print(_from,_to,_cc,_bcc)
    global finald
    if _from != []:
        if _to != []:
            for each in _to:
                finald.append((_from[0], each))
        if _cc != []:
            for each in _cc:
                finald.append((_from[0], each))
        if _bcc != []:
            for each in _bcc:
                finald.append((_from[0], each))

In [6]:
def find_from_to_cc(file):
    __from = []
    __to = []
    __cc = []
    __bcc = []
    with open(file, 'r') as f:
        #print(file)
        if '.DS_Store' in file:
            return __from, __to, __cc, __bcc
#         print(file)
        try:
            a = str(f.read())
            f.close()
            _from = re.findall("X-From: ([\s\S]*)\nX-To", a)
            _to = re.findall("X-To: ([\s\S]*)\nX-cc", a)
            _cc = re.findall("X-cc: ([\s\S]*)\nX-bcc", a)
            _bcc = re.findall("X-bcc: ([\s\S]*)\nX-Folder", a)
            #处理to cc bcc内多个人员的情况
            _to_ = _to[0].split(', ')
            _cc_ = _cc[0].split(', ')
            _bcc_ = _bcc[0].split(', ')

            #将名称对应起来
            #发件人的名称比较规范
            from_name = _from[0].split(' ')
            xing = from_name[-1]
            ming = from_name[0]
            if xing in m.xing and ming in m.ming:
                #print(from_name)
                __from.append(_from[0])
                #print('from',_from)
            for each in _to_:
                to_name = each.split(' ')
                x = to_name[-1]
                mm = to_name[0]
                if x in m.xing and mm in m.ming:
                    # print('------',each,'-------')
                    __to.append(each)
            for each in _cc_:
                cc_name = each.split(' ')
                x = cc_name[-1]
                mm = cc_name[0]
                if x in m.xing and mm in m.ming:
                    # print('------',each,'-------')
                    __cc.append(each)
            for each in _bcc_:
                bcc_name = each.split(' ')
                x = bcc_name[-1]
                mm = bcc_name[0]
                if x in m.xing and mm in m.ming:
                    # print('------',each,'-------')
                    __bcc.append(each)
            # print('from', __from)
            # print('to',__to)
            # print('cc',__cc)
            # print('bcc',__bcc)
            #print(__from, __to, __cc, __bcc)
        except:
            None
        return __from, __to, __cc, __bcc

In [7]:
finald = []
file_path('./maildir/allen-p')
pp = Counter(finald)
print(pp)

Counter({('Phillip K Allen', 'Keith Holst'): 156, ('Phillip K Allen', 'Mike Grigsby'): 131, ('Phillip K Allen', 'Frank Ermis'): 76, ('Phillip K Allen', 'John J Lavorato'): 63, ('Phillip K Allen', 'Matthew Lenhart'): 51, ('Phillip K Allen', 'Hunter S Shively'): 30, ('Phillip K Allen', 'Jay Reitmeyer'): 28, ('Phillip K Allen', 'Tori Kuykendall'): 24, ('Phillip K Allen', 'Jane M Tholt'): 20, ('Phillip K Allen', 'Robert Badeer'): 19, ('Phillip K Allen', 'Randall L Gay'): 16, ('Phillip K Allen', 'Cooper Richey'): 14, ('Phillip K Allen', 'James D Steffes'): 13, ('Phillip K Allen', 'Fletcher J Sturm'): 12, ('Phillip K Allen', 'Steven J Kean'): 9, ('Phillip K Allen', 'David W Delainey'): 8, ('Phillip K Allen', 'Paul T Lucci'): 8, ('Phillip K Allen', 'Barry Tycholiz'): 7, ('Phillip K Allen', 'Andy Zipper'): 7, ('Phillip K Allen', 'Susan M Scott'): 7, ('Phillip K Allen', 'Thomas A Martin'): 6, ('Phillip K Allen', 'Jeff Dasovich'): 5, ('Phillip K Allen', 'Richard B Sanders'): 5, ('Phillip K Allen

    如上面程序运行的结果展示的，allen-p这个人所有的邮件发送关系我们都已经掌握了。其中有部分邮件并不是由这个人所发送，可能是存在转存的原因，但是获取到的结果集合一定是我们需要的allen-p目标集合的一个超集。
    

### 接下来是一步非常重要的处理工作，用来修补上面收件人格式“姓”“名”匹配原则的漏洞
    仔细思考，这个匹配原则是否存在漏洞？
|  姓|名|
|:-:|:-:|
|王|凌|
|吴|彦祖|
|张|学友|
|陈|奕迅|
    
    假设这四个人是公司的所有高管，已经构造出了“姓”“名”这两个列表，根据我们的匹配原则，如果出现了这么几个人
|  姓|名|
|:-:|:-:|
|王|彦祖|
|王|学友|
|王|奕迅|

    这两个人并不是公司的高管，但是也能够通过我们的匹配规则！为了解决这个问题，需要对发件人和高管名单再次进行一轮匹配，进一步剔除非高管人员。同时要注意，Phillip K Allen和Phillip Allen是同一个人，这种情况需要进行保留。
    通过上面的操作，得到了一组正确无冗余的边的关系集合，将其保存到csv中。
```python
#无向图
f = open('result.csv','a+')
    for k,v in pp.items():
        f.write(str(k[0])+','+str(k[1])+','+str(v)+'\n')
    f.close()
#有向图
f = open('result2.csv','a+')
    for k,v in pp.items():
        f.write(str(k[0])+','+str(k[1])+','+str(v)+'\n')
    f.close()
```

    为了方便Gephi输入，给这份边的关系加上一个header。在这里，生成了两份csv，一份作为有向图（上）的边关系，一份作为无向图（下）的边关系。

![有向图](./pics/6.png "有向图")

---
![无向图](./pics/7.png "无向图")

### 2.4一张等权重的无向通联关系图

    在构建出点（new_member.csv）和无向边(result.csv)这两个图的基本要素之后，就可以开始进行图的可视化。
    输入点和无向边
![点](./pics/8.png "点")
![无向边](./pics/8.png "无向边")
    
    在Gephi内进行算法和参数调整，输入一个无向的通联关系图。
![基础](./pics/基础.svg)

---

#### 如何评价这张无向图？
* 评价一个节点的重要性
    > 节点的大小
    > 节点的度
* 评价一条边的重要性
    > 与节点相连接的边的权重
 
---    
        
        对于一个节点，其大小在一定程度上反映了这个节点在网络中的重要性，这里节点label的大小和节点的重要性呈线性关系，因此，label越大，这个节点在网络中就越重要。
        John J Lavorato，作为安然公司的CEO，在整个网络中占了最重要的地位。其次Louise Kitchen和David W Delainey，作为President和另一位CEO，在网络中也有着举足轻重的地位。
        但是在网络中也有一些比较特殊的情况，例如同样身为CEO的Jeff Skilling，他所代表的节点并没有体现出非常明显的特征。实际上，这也是可以理解的，毕竟网络的构建来源于数据集，一方面，可能Jeff Skilling本身使用邮件的次数较少，因此留下的原始数据较少，另一方面，可能在提取通联关系的过程中又丢失了一部分数据。
        对于边来说，在图中，可以明显看到存在若干条颜色较深、宽度较宽的边存在。这些边意味着两个节点之间的联系十分频繁。由于这是一张无向图，因此并不能够确定是双方相互来往，还是只有单方面发邮件较多。但是无论如何，这样的边始终意味着一种较为紧密的联系。

        利用Gephi的模块化功能，可以很方便地得到这样一个社团发现的结果。可以看到，在安然高层管理者中，总是用几位“大佬”真正负责着串联整个公司的作用，亦即这些节点是网络中十分关键的节点，各自位于一个社团的中央，和本社团其它节点、其它社团的节点都保持着频繁的联系。

### 2.5基于Pagerank算法的有向图建立
        建立一张有向图，同样需要点和边的数据，在2.3中已经构建完成。下面就是利用Pagerank算法，对网络中的节点进行权值的度量。
        首先读取节点和边的关系，构造转移矩阵。
```python
#首先读取点和边
path = './result2的副本.csv'
path_menber = './new_member.csv'
member = pd.read_csv(path_menber)['id']
#将人物姓名排序，方便对应后面的权值
member = set(member)
member = list(member)
member.sort()
n = len(member)
#建立初始转移矩阵，初始化大小为n*n，全0
target = np.zeros((n, n))
target = np.array(target)
route = pd.read_csv(path,header=None)
route = np.array(route)
'''
route就是一组邮件的收发关系，包括三个元素(from, to, weight)
根据转移矩阵的定义，根据收发人的index和weight调整转移矩阵相应元素的值
'''
for each in route:
    from_ = each[0]
    to_ = each[1]
    weight = each[2]
    k1 = member.index(from_)
    k2 = member.index(to_)
    target[k2][k1] = weight
print(target)
```

In [8]:
import pandas as pd
import numpy as np
path = './result2的副本.csv'
path_menber = './new_member.csv'
member = pd.read_csv(path_menber)['id']
member = set(member)
member = list(member)
member.sort()
n = len(member)
target = np.zeros((n, n))
target = np.array(target)
route = pd.read_csv(path,header=None)
route = np.array(route)
for each in route:
    from_ = each[0]
    to_ = each[1]
    weight = each[2]
    k1 = member.index(from_)
    k2 = member.index(to_)
    target[k2][k1] = weight
print(target)

[[0.00e+00 0.00e+00 0.00e+00 ... 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 ... 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 ... 0.00e+00 4.00e+00 0.00e+00]
 ...
 [0.00e+00 0.00e+00 0.00e+00 ... 0.00e+00 5.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 ... 0.00e+00 4.97e+03 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 ... 0.00e+00 0.00e+00 0.00e+00]]


    输出每一列之和

In [9]:
row_sum = list(target.sum(axis=0))
print(row_sum)

[147.0, 12.0, 76.0, 139.0, 43.0, 6.0, 193.0, 82.0, 1321.0, 9.0, 426.0, 58.0, 3.0, 34.0, 14.0, 1855.0, 12.0, 353.0, 1453.0, 434.0, 1296.0, 2.0, 111.0, 126.0, 19.0, 21.0, 608.0, 90.0, 13.0, 9.0, 158.0, 22.0, 3485.0, 166.0, 0.0, 53.0, 9939.0, 12.0, 0.0, 331.0, 89.0, 0.0, 0.0, 5.0, 741.0, 23.0, 195.0, 1263.0, 2.0, 23.0, 53.0, 34.0, 434.0, 261.0, 0.0, 0.0, 88.0, 96.0, 25.0, 64.0, 50.0, 58.0, 154.0, 155.0, 460.0, 24.0, 172.0, 564.0, 895.0, 205.0, 87.0, 891.0, 666.0, 2.0, 123.0, 173.0, 28.0, 0.0, 3.0, 75.0, 34.0, 9.0, 70.0, 1032.0, 503.0, 12.0, 96.0, 0.0, 629.0, 1172.0, 121.0, 14.0, 4.0, 107.0, 0.0, 577.0, 36.0, 843.0, 913.0, 99.0, 120.0, 628.0, 2576.0, 191.0, 18.0, 902.0, 4746.0, 0.0, 38.0, 27.0, 10.0, 160.0, 32.0, 5715.0, 0.0]


    将每一列的和调整为1

In [10]:
for col in range(n):
    if row_sum[col] != 0:
        for row in range(n):
            target[row][col] = target[row][col]/row_sum[col]
print(target.sum(axis=0))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0.]


    可以看到，在转移矩阵中，有若干列的和为0。也就是说，这个矩阵不是随机矩阵，不满足收敛的条件。为了让转移矩阵收敛，采用抽税法。
$$v^{'}={\beta}Mv+(1-{\beta})e/n$$

    从公式中可以看出，除了转移矩阵M，还需要一个向量e和阻尼系数，向量v的大小是n*1，其初始值全为1/n。
    
```python
def pagerank(n,transmatrix):
    #生成概率
    beta = 0.8
    x = np.ones((n,1))/n
    p = np.ones((n,1))/n
    for i in range(1000):
        if x.all == (beta*np.dot(transmatrix, x)+(1-beta)*p).all:
            print(i)
            break
        x = beta*np.dot(transmatrix, x)+(1-beta)*p
        print(x.sum(axis = 0))
    return x
```
    从理论上说，Pagerank算法迭代结束的条件是前后向量v不变，但是在实际操作中，v保持不变几乎是不可能的，而且在测试中，一般迭代20次左右，v的值就不会有太大的变化，因此就取这时候的v作为最终的向量。

In [11]:
def pagerank(n,transmatrix):
    #生成概率
    beta = 0.8
    x = np.ones((n,1))/n
    p = np.ones((n,1))/n
    for i in range(30):
        if x.all == (beta*np.dot(transmatrix, x)+(1-beta)*p).all:
            print(i)
            break
        x = beta*np.dot(transmatrix, x)+(1-beta)*p
        print(x.sum(axis = 0))
    return x

In [12]:
x = pagerank(n, target)

[0.92347826]
[0.90602698]
[0.89399909]
[0.8851917]
[0.87822917]
[0.87311115]
[0.86921147]
[0.86627304]
[0.86402978]
[0.86232016]
[0.86100958]
[0.86000361]
[0.85922895]
[0.85863136]
[0.85816939]
[0.85781168]
[0.85753429]
[0.85731891]
[0.85715149]
[0.85702122]
[0.85691978]
[0.85684072]
[0.85677906]
[0.85673096]
[0.85669341]
[0.85666408]
[0.85664118]
[0.85662327]
[0.85660928]
[0.85659834]


    接着按照（姓名-权重）的方式，输出每个高管（节点）对应的权值。

In [13]:
for a,b in zip(member, x):
    print(str(a)+','+str(b[0]))

Andrea Ring,0.002728605116871463
Andrew Lewis,0.0033074412124809074
Andy Zipper,0.007116659900528143
Barry Tycholiz,0.008581661835954206
Benjamin Rogers,0.0029937398996815693
Bill Rapp,0.0017391304347826083
Cara Semperger,0.008641095472192469
Chris Dorland,0.0035509576084816943
Chris Germany,0.008995687920789894
Cooper Richey,0.002688318732481878
Dan Hyvl,0.013226523760780632
Dana Davis,0.010725000451209705
Danny McCarty,0.003678327435693773
Daren Farmer,0.003296468549645668
Darrell Schoolcraft,0.0032723768162083483
David Delainey,0.01355757515436347
Diana Scholtes,0.006667184350229421
Don Baughman,0.008766699634833373
Drew Fossum,0.007386435088759931
Elizabeth Sager,0.010725373077700193
Eric Bass,0.007859846734066653
Eric Saibi,0.0018684334525537261
Errol McLaughlin,0.0038807748034928105
Fletcher Sturm,0.004479017628270733
Frank Ermis,0.007361172179368317
Geir Solberg,0.002045029270676544
Gerald Nemec,0.015163488299863219
Greg Whalley,0.010344076112902586
Greg Wolfe,0.0062551521222374

    在Gephi中，利用刚才获得的有向图边和之前的点数据构建有向图。
![有向图](./pics/基础2.svg)

    有向图的结论和无向图基本上是一致的，都可以看出几位特定的高管占据了网络的关键位置。

### 2.6人以群分——社团发现
#### 利用Gephi进行社团发现
    话不多说，直接上图
##### 无向图的社团发现
![社团发现](./pics/社团.svg)
##### 有向图的社团发现
![社团发现](./pics/有向社团.svg)

### 2.7总结